In [ ]:
from flask import Flask, render_template, request
import keras
import tensorflow
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from PIL import Image
import pickle

app = Flask(__name__)

path = r'D:\Projects\eyewear_classification'
vgg_model = VGG16()
vgg_model = Model(inputs = vgg_model.inputs, outputs = vgg_model.layers[-2].output)
svm_model1 = pickle.load(open(path+r'/svm_model1.sav', 'rb'))
svm_model2 = pickle.load(open(path+r'/svm_model2.sav', 'rb'))
pca = pickle.load(open(path+r'/pca.sav', 'rb'))


def extract_features(model,im_path):
    # load the image as a 224x224 array
    img = load_img(im_path, target_size=(224,224))
    img = np.array(img)
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3)
    imgx = preprocess_input(reshaped_img)
    features = model.predict(imgx, use_multiprocessing=True)
    features = pca.transform(features)
    return features

def pred_type(im_feat):
    im_feat = pd.DataFrame(im_feat)
    pt = svm_model1.predict(im_feat)
    return pt

def pred_shape(im_feat):
    im_feat = pd.DataFrame(im_feat)
    ps = svm_model2.predict(im_feat)
    return ps

def pred_sim_imgs(x,filenames,im_feat):
    kmeans = KMeans(n_clusters=16, random_state=22)
    kmeans.fit(x)
    groups = {}
    # grouping the images belonging to same cluster
    for file, cluster in zip(filenames,kmeans.labels_):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(file)
        else:
            groups[cluster].append(file)
    # extracting features all members of the cluster to which our sample belongs
    clust_ind = kmeans.labels_[0]
    cluster_x = x[np.where(kmeans.labels_== clust_ind)]
    # calculating eucilidian distance between feature vector of our sample and every member of it's cluster
    distances = np.linalg.norm(cluster_x - im_feat, axis=1)
    k = 11
    # ids contains the index of 11 closest members of the cluster to the test image
    ids = distances.argsort()[:k]
    mat = []
    for i in ids:
        mat.append(groups[clust_ind][i])
    # Retrieving url(s)
    u1 = mat[0]
    u2 = mat[1]
    u3 = mat[2]
    u4 = mat[3]
    u5 = mat[4]
    u6 = mat[5]
    u7 = mat[6]
    u8 = mat[7]
    u9 = mat[8]
    u10 = mat[9]
    return u1,u2,u3,u4,u5,u6,u7,u8,u9,u10

@app.route("/", methods=['GET','POST'])
def main():
    return render_template("index.html")

@app.route("/submit", methods=['GET','POST'])
def get_output():
    if request.method == 'POST':
        img = request.files['my_image']
        
        img_path = r'static/'+ img.filename
        img.save(img_path)
        
        im_feat = extract_features(vgg_model,img_path)
        pt = pred_type(im_feat)
        ps = pred_shape(im_feat)
        filenames = np.load(path+r'\filenames.npy')
        filenames = np.concatenate((np.array([img_path]),filenames),axis=0)
        x = np.load(path+r'\x.npy')
        x = np.concatenate((im_feat, x), axis=0)
        u1,u2,u3,u4,u5,u6,u7,u8,u9,u10 = pred_sim_imgs(x,filenames,im_feat)
    return render_template("index.html", prediction_type = pt, prediction_shape = ps,img_path=img_path, img_path1 = u1, img_path2 = u2, img_path3 = u3, img_path4 = u4, img_path5 = u5, img_path6 = u6, img_path7 = u7, img_path8 = u8,img_path9 = u9, img_path10 = u10)


if __name__ == '__main__':
    #app.run(debug=True, use_reloader=False)
    app.run(debug=False)